In [1]:
import os
import math
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
from matplotlib import cm
from scipy import stats
from scipy import optimize
from scipy import integrate
import numpy.polynomial.polynomial as poly
from bisect import bisect_left
import pprint

In [132]:
def import_file(path, limit_low=None, limit_high=None):

    spectrum = np.genfromtxt(path, delimiter=",")
    spectrum = np.transpose(spectrum)
    wavenumbers = spectrum[0]
    intensities = spectrum[1]

    if limit_low is not None:
        limit_low_index = list(wavenumbers).index(limit_low)
    else:
        limit_low_index = 0
        limit_low = wavenumbers[0]

    if limit_high is not None:
        limit_high_index = list(wavenumbers).index(limit_high)
    else:
        limit_high_index = len(wavenumbers)
        limit_high = wavenumbers[-1]

    wavenumbers = wavenumbers[limit_low_index:limit_high_index]
    intensities = intensities[limit_low_index:limit_high_index]
    return wavenumbers, intensities

def import_directory(path, limit_low=None, limit_high=None):
    # files = os.listdir(path)

    # for filename in files:
    #     np.genfromtxt(filename, delimiter=",")
    pass

In [157]:
wavenumbers, intensities = import_file("spectra/E (1).TXT", 300, 1600)

In [158]:
intensities_sg0 = sig.savgol_filter(intensities, 
                                    window_length=13, 
                                    polyorder=3, 
                                    deriv=0)

In [159]:
?sig.argrelmin

Signature: sig.argrelmin(data, axis=0, order=1, mode='clip')
Docstring:
Calculate the relative minima of `data`.

Parameters
----------
data : ndarray
    Array in which to find the relative minima.
axis : int, optional
    Axis over which to select from `data`. Default is 0.
order : int, optional
    How many points on each side to use for the comparison
    to consider ``comparator(n, n+x)`` to be True.
mode : str, optional
    How the edges of the vector are treated.
    Available options are 'wrap' (wrap around) or 'clip' (treat overflow
    as the same as the last (or first) element).
    Default 'clip'. See numpy.take.

Returns
-------
extrema : tuple of ndarrays
    Indices of the minima in arrays of integers. ``extrema[k]`` is
    the array of indices of axis `k` of `data`. Note that the
    return value is a tuple even when `data` is 1-D.

See Also
--------
argrelextrema, argrelmax, find_peaks

Notes
-----
This function uses `argrelextrema` with np.less as comparator. Therefor

In [160]:
## Basislinie Versuch 3 

# Liste an Minima erstellen
mins = list(sig.argrelmin(intensities_sg0, order=5)[0])
mins.insert(0, 0)
mins.append(len(wavenumbers)-1)

intervals = {}
for i in range(len(mins)-1):
    interval = (mins[i], mins[i+1]) # Immer 2 benachbarte Minima sind ein Intervall
    
    # Ein Minimum davor und danach hinzufügen -> für Polynom
    if i == 0:
        baseline_points = [0] + mins[:3]
        baseline_wns = wavenumbers[baseline_points]
        baseline_wns[0] = 2*baseline_wns[1] - baseline_wns[2]
    elif i == len(mins)-2:
        baseline_points = mins[-3:] + [mins[-1]]
        baseline_wns = wavenumbers[baseline_points]
        baseline_wns[-1] = 2*baseline_wns[-2] - baseline_wns[-3]
    else:
        baseline_points = mins[i-1:i+3]
        baseline_wns = wavenumbers[baseline_points]
    baseline_ints = intensities_sg0[baseline_points]
    
    # Polynom fitten und Basislinie berechnen
    interval_relative = [i-baseline_points[0] for i in interval]
    fit = poly.polyfit(baseline_wns, baseline_ints, 3)
    x = wavenumbers[baseline_points[0]:baseline_points[-1]]
    local_baseline = poly.polyval(x, fit)
    
    wns = wavenumbers[interval[0]:interval[1]]
    ints = intensities_sg0[interval[0]:interval[1]]
    ints_corrected = ints - local_baseline[interval_relative[0]:interval_relative[1]]
    
    # Negative Bereiche korrigieren
    mins_new = sig.argrelmin(ints_corrected)[0]
    
    mins_new = [0] + [x for x in mins_new if ints_corrected[x] < 0] + [len(ints_corrected)-1]
    
    baseline2_wns = wns[mins_new]
    baseline2_ints = ints_corrected[mins_new]
    fit2 = poly.polyfit(baseline2_wns, baseline2_ints, len(mins_new)-1)
    local_baseline2 = poly.polyval(x, fit2)
    ints_corrected = ints_corrected - local_baseline2[interval_relative[0]:interval_relative[1]]
    local_baseline += local_baseline2
    
    intervals[interval] = {"wavenumbers":wns,
                           "intensities":ints,
                           "baseline_wns":x,
                           "baseline_ints":local_baseline,
                           "corrected_ints":ints_corrected,
                           "rel_int":interval_relative}


In [161]:
## Basislinie Versuch 3 

def calculate_baseline(wavenumbers, intensities):

    # Liste an Minima erstellen
    mins = list(sig.argrelmin(intensities, order=5)[0])
    mins.insert(0, 0)
    mins.append(len(wavenumbers)-1)

    intervals = {}
    for i in range(len(mins)-1):
        interval = (mins[i], mins[i+1]) # Immer 2 benachbarte Minima sind ein Intervall

        # Ein Minimum davor und danach hinzufügen -> für Polynom
        if i == 0:
            baseline_points = [0] + mins[:3]
            baseline_wns = wavenumbers[baseline_points]
            baseline_wns[0] = 2*baseline_wns[1] - baseline_wns[2]
        elif i == len(mins)-2:
            baseline_points = mins[-3:] + [mins[-1]]
            baseline_wns = wavenumbers[baseline_points]
            baseline_wns[-1] = 2*baseline_wns[-2] - baseline_wns[-3]
        else:
            baseline_points = mins[i-1:i+3]
            baseline_wns = wavenumbers[baseline_points]
        baseline_ints = intensities[baseline_points]

        # Polynom fitten und Basislinie berechnen
        interval_relative = [i-baseline_points[0] for i in interval]
        fit = poly.polyfit(baseline_wns, baseline_ints, 3)
        x = wavenumbers[baseline_points[0]:baseline_points[-1]]
        local_baseline = poly.polyval(x, fit)

        wns = wavenumbers[interval[0]:interval[1]]
        ints = intensities[interval[0]:interval[1]]
        ints_corrected = ints - local_baseline[interval_relative[0]:interval_relative[1]]

        # Negative Bereiche korrigieren
        mins_new = sig.argrelmin(ints_corrected)[0]

        mins_new = [0] + [x for x in mins_new if ints_corrected[x] < 0] + [len(ints_corrected)-1]

        baseline2_wns = wns[mins_new]
        baseline2_ints = ints_corrected[mins_new]
        fit2 = poly.polyfit(baseline2_wns, baseline2_ints, len(mins_new)-1)
        local_baseline2 = poly.polyval(x, fit2)
        ints_corrected = ints_corrected - local_baseline2[interval_relative[0]:interval_relative[1]]
        local_baseline += local_baseline2

        intervals[interval] = {"wavenumbers":wns,
                               "intensities":ints,
                               "baseline_wns":x,
                               "baseline_ints":local_baseline,
                               "corrected_ints":ints_corrected,
                               "rel_int":interval_relative}
        
    return intervals

In [162]:
intervals = calculate_baseline(wavenumbers, intensities_sg0)

In [163]:
## Peakfinding

for i, d in intervals.items():
    print(i)
    
    try:
        intensities_sg2 = sig.savgol_filter(d["corrected_ints"], 
                                            window_length=13, 
                                            polyorder=3, 
                                            deriv=2)
    except ValueError:
        intensities_sg2 = np.zeros(len(d["corrected_ints"]))

    plt.plot(d["wavenumbers"], intensities_sg2)
    peaks = sig.argrelmin(intensities_sg2, order=10)[0]
    peaks = [peak for peak in peaks if intensities_sg2[peak] < 0]
    
    peak_condensing = []
    peaks_new = []
    for i in range(len(intensities_sg2)):
        if i in peaks:
            peak_condensing.append(i)
        if intensities_sg2[i] > 0 and len(peak_condensing) > 0:
            peaks_new.append(int(np.mean(peak_condensing)))
            peak_condensing = []
    if len(peak_condensing) > 0:
        peaks_new.append(int(np.mean(peak_condensing)))
    peaks = peaks_new
    
    valleys = [0]
    
    for i in range(len(peaks)-1):
        valleys.append(peaks[i] + np.argmax(intensities_sg2[peaks[i]:peaks[i+1]]))
    
    valleys.append(len(intensities_sg2)-1)
    
    peak_intervals = []
    for i in range(len(peaks)):
        peak_intervals.append((valleys[i], valleys[i+1]))
        
    peak_heights = d["corrected_ints"][peaks]
    peak_curvature = - intensities_sg2[peaks]
    peak_score = [height*curvature for height, curvature in zip(peak_heights, peak_curvature)]
    peak_indices = list(range(len(peaks)))
    
    peaks = [peak for _,peak in sorted(zip(peak_heights,peaks))]
    peak_intervals = [interval for _,interval in sorted(zip(peak_heights,peak_intervals))]
    peak_indices = [index for _,index in sorted(zip(peak_heights, peak_indices))]
    peaks.reverse()
    peak_intervals.reverse()
    peak_indices.reverse()
    peak_wns = d["wavenumbers"][peaks]
    peak_heights = d["corrected_ints"][peaks]
    
    d["peaks"] = peaks
    d["peak_wns"] = peak_wns
    d["peak_heights"] = peak_heights
    d["peak_intervals"] = peak_intervals
    d["peak_indices"] = peak_indices

(0, 27)
(27, 310)
(310, 369)
(369, 498)
(498, 594)
(594, 796)
(796, 947)
(947, 983)
(983, 1006)
(1006, 1190)
(1190, 1261)
(1261, 1392)
(1392, 1453)
(1453, 1548)
(1548, 1729)
(1729, 1934)
(1934, 1966)
(1966, 2218)
(2218, 2261)
(2261, 2389)
(2389, 2408)
(2408, 2599)


In [164]:
%matplotlib
for i, d in intervals.items():
    fig, ax = plt.subplots(2,1)
    fig.suptitle(i)
    ax[0].plot(d["wavenumbers"], d["intensities"], linewidth=1, color="blue")
    ax[0].plot(d["baseline_wns"], d["baseline_ints"], linewidth=1, color="red")
    ax[0].fill_between(d["wavenumbers"],
                       d["intensities"],
                       d["baseline_ints"][d["rel_int"][0]:d["rel_int"][1]])
    ax[1].plot(d["wavenumbers"], d["corrected_ints"], linewidth=1, color="blue")
    ax[1].hlines(0, d["wavenumbers"][0], d["wavenumbers"][-1])
    ax[1].fill_between(d["wavenumbers"],
                       d["corrected_ints"],
                       0)
    if len(d["peak_heights"]) > 0:
        ax[1].vlines(d["peak_wns"], 0, d["peak_heights"][0]*1.1, color="red")
    
    #ax.vlines(values["wavenumbers"][values["peaks"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="red")
    #ax.vlines(values["wavenumbers"][values["valleys"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="blue")

Using matplotlib backend: Qt5Agg


<ipython-input-164-23ab3d095a3f>:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(2,1)


In [165]:

def gaussian(x, amplitude, mean, stddev):
    return (amplitude * np.exp(-((x - mean)**2 / (2 * stddev**2))))

for i, d in intervals.items():
    print(i)
    corrected_ints = d["corrected_ints"][:]
    gauss_all = []
    gauss_cumulative = []
    params_all = []
    for j in range(len(d["peaks"])):
        peak = d["peaks"][j]
        peak_interval = d["peak_intervals"][j]
        print(peak_interval)
        index = d["peak_indices"][j]
        p0 = (corrected_ints[peak], d["wavenumbers"][peak], d["wavenumbers"][peak_interval[1]] - d["wavenumbers"][peak_interval[0]])
        try:
            p = optimize.curve_fit(gaussian,
                                   d["wavenumbers"][peak_interval[0]:peak_interval[1]],
                                   corrected_ints[peak_interval[0]:peak_interval[1]],
                                   p0=p0, method="lm")[0]
        except RuntimeError:
            p = [0, d["wavenumbers"][peak], 1]
        print(p)
        params_all.append(p)
        gauss = []
        for wn in d["wavenumbers"]:
            gauss.append(gaussian(wn, p[0], p[1], p[2]))
        gauss = np.array(gauss)
        corrected_ints = corrected_ints - gauss
        gauss_all.append(gauss)
        if j == 0:
            gauss_stacked = gauss
        else:
            gauss_stacked = gauss + gauss_cumulative[j-1]
        
        gauss_cumulative.append(gauss_stacked)
        
        if index != 0:
            j_prev = d["peak_indices"].index(index-1)
            d["peak_intervals"][j_prev] = (d["peak_intervals"][j_prev][0], peak_interval[1])
        
        if index != len(d["peaks"])-1:
            j_next = d["peak_indices"].index(index+1)
            d["peak_intervals"][j_next] = (peak_interval[0], d["peak_intervals"][j_next][1])
        
    d["gauss"] = gauss_all
    d["gauss_cum"] = gauss_cumulative
    d["params"] = params_all
    

(0, 27)
(27, 310)
(0, 37)
[1970.76462762  323.5866377    -4.83915078]
(101, 175)
[1227.88980915  383.81176724   14.85611316]
(101, 282)
[628.10612828 422.86471812 -10.77733795]
(0, 175)
[164.64442779 335.32838721  -3.23438689]
(310, 369)
(0, 41)
[127.67767718 466.76690767  -4.91538592]
(0, 58)
[  5.24996995 479.93894163   1.76400756]
(369, 498)
(0, 128)
[1868.0083451   518.53127075   11.19293638]
(498, 594)
(0, 95)
[1505.49886013  571.85410198   -8.75397664]
(594, 796)
(86, 201)
[1.19213054e+04 6.57133509e+02 1.14772973e+01]
(33, 201)
[1366.2862725   624.99300214    9.2938637 ]
(0, 201)
[188.20761672 609.07958348   4.09595473]
(796, 947)
(0, 73)
[5090.51478132  725.72989396   10.08278598]
(0, 150)
[2495.82065746  745.8331145     6.63410613]
(947, 983)
(0, 35)
[ 69.3889652  781.24831526  -4.11052575]
(983, 1006)
(0, 22)
[ 33.54570098 798.07334019  -2.38973156]
(1006, 1190)
(73, 109)
[1157.78187403  846.89424242   11.33560304]
(0, 109)
[1029.55660151  823.34861207    7.79925545]
(73, 183

In [166]:
## Peak quality control
peaks_removed = []

for i, d in intervals.items():
    for j in range(len(d["peaks"])):
        gauss_amp = d["params"][j][0]
        if gauss_amp <= 0:
            peaks_removed.append((i, j))
            continue
            
        gauss_mean = d["params"][j][1]
        peak_interval = d["peak_intervals"][j]
        if  not (d["wavenumbers"][peak_interval[0]] < gauss_mean < d["wavenumbers"][peak_interval[1]]):
            peaks_removed.append((i, j))
            continue

pprint.pprint(peaks_removed)

[((1729, 1934), 3), ((2261, 2389), 2), ((2408, 2599), 2)]


In [167]:
area_sum = 0
area_threshold = 10
areas_all = []

for i, d in intervals.items():
    areas = []
    for j in range(len(d["peaks"])):
        x = d["wavenumbers"]
        y = d["gauss"][j]

        area = integrate.simpson(y, x=x)
        
        if area < area_threshold:
            peaks_removed.append((i, j))
        
        if (i, j) in peaks_removed:
            area = 0
        else:
            areas_all.append(area)
        
        area_sum += area
        areas.append(area)
    d["areas"] = areas

print(area_sum)
print(np.median(areas_all))
print(np.mean(areas_all))

1487366.4263716275
20036.30387263211
36277.229911503106


In [168]:
for i, d in intervals.items():
    print(i)
    print(d["peak_wns"])
    print(d["areas"])
    print()

(0, 27)
[]
[]

(27, 310)
[323. 385. 425. 357.]
[23461.531513134345, 45724.99040079508, 16939.882508916115, 1334.8391613514002]

(310, 369)
[467. 481.]
[1559.6519535473506, 22.966202036500533]

(369, 498)
[518.5]
[52153.5350032983]

(498, 594)
[571.]
[32805.43146029581]

(594, 796)
[659.5 621.  607.5]
[342892.9845856551, 31788.064321219055, 1929.262081237273]

(796, 947)
[724.  743.5]
[128273.34919971683, 41502.71970835232]

(947, 983)
[779.5]
[687.4057685510714]

(983, 1006)
[798.5]
[193.9189451932044]

(1006, 1190)
[848.5 824.  872. ]
[32895.16686590996, 20036.30387263211, 13716.160985508677]

(1190, 1261)
[906.5 924.5]
[3983.5699986815043, 336.2077750310844]

(1261, 1392)
[958.5 985.5]
[60522.80291447349, 2926.426857898894]

(1392, 1453)
[1002.  1018.5]
[835.55555900777, 209.07186913936013]

(1453, 1548)
[1052.]
[28854.57902407567]

(1548, 1729)
[1135.  1095.5]
[76944.18401674871, 22244.04173914164]

(1729, 1934)
[1244. 1213. 1196. 1176.]
[65823.79225855047, 43529.87749715953, 6504.5

In [145]:
%matplotlib
for i, d in intervals.items():
    fig, ax = plt.subplots(2,1)
    fig.suptitle(i)
    ax[0].plot(d["wavenumbers"], d["intensities"], linewidth=1, color="blue")
    ax[0].plot(d["baseline_wns"], d["baseline_ints"], linewidth=1, color="red")
    ax[0].fill_between(d["wavenumbers"],
                       d["intensities"],
                       d["baseline_ints"][d["rel_int"][0]:d["rel_int"][1]])
    ax[1].plot(d["wavenumbers"], d["corrected_ints"], linewidth=1, color="blue")
    for j in range(len(d["peaks"])):
        if (i, j) in peaks_removed:
            ax[1].plot(d["wavenumbers"], d["gauss_cum"][j], linewidth=1, linestyle="--", color="red")
        else:
            ax[1].plot(d["wavenumbers"], d["gauss_cum"][j], linewidth=1, color="black")
    ax[1].hlines(0, d["wavenumbers"][0], d["wavenumbers"][-1])
    ax[1].fill_between(d["wavenumbers"],
                       d["corrected_ints"],
                       0)
    if len(d["peak_heights"]) > 0:
        ax[1].vlines(d["peak_wns"], 0, d["peak_heights"][0]*1.1, color="red")
    
    #ax.vlines(values["wavenumbers"][values["peaks"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="red")
    #ax.vlines(values["wavenumbers"][values["valleys"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="blue")
    

Using matplotlib backend: Qt5Agg


In [49]:
?list.index

Signature: list.index(self, value, start=0, stop=9223372036854775807, /)
Docstring:
Return first index of value.

Raises ValueError if the value is not present.
Type:      method_descriptor
